In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [10]:
class Time2Vec(tf.keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        super(Time2Vec, self).__init__(**kwargs)
        self.output_dim = output_dim

    def build(self, input_shape):
        self.w = self.add_weight(shape=(2, self.output_dim),  # Modified shape to (2, output_dim)
                                 initializer='uniform',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.output_dim,),  # Removed input_shape[1]
                                 initializer='zeros',
                                 trainable=True)
        super(Time2Vec, self).build(input_shape)

    def call(self, x):
        time_values = tf.math.floormod(x[:, 0], 24)  # Assuming time values range from 0 to 24
        sin_trans = tf.math.sin(tf.matmul(tf.ones_like(time_values), self.w) + self.b)  # Modified tf.matmul input
        cos_trans = tf.math.cos(tf.matmul(tf.ones_like(time_values), self.w) + self.b)  # Modified tf.matmul input
        return tf.concat([sin_trans, cos_trans], axis=-1)

# Rest of the code remains the same


In [3]:
class Time2Vec(tf.keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        super(Time2Vec, self).__init__(**kwargs)
        self.output_dim = output_dim

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[1], self.output_dim),
                                 initializer='uniform',
                                 trainable=True)
        self.b = self.add_weight(shape=(input_shape[1],),
                                 initializer='zeros',
                                 trainable=True)
        super(Time2Vec, self).build(input_shape)

    def call(self, x):
        time_values = tf.math.floormod(x[:, 0], 24)  # Assuming time values range from 0 to 24
        sin_trans = tf.math.sin(tf.matmul(time_values, self.w) + self.b)
        cos_trans = tf.math.cos(tf.matmul(time_values, self.w) + self.b)
        return tf.concat([sin_trans, cos_trans], axis=-1)

In [11]:
# Define an example model with Time2Vec layer
input_shape = (1,)  # Single time value as input
output_dim = 32  # Dimension of the Time2Vec output
inputs = Input(shape=input_shape)
time2vec = Time2Vec(output_dim)(inputs)

ValueError: Exception encountered when calling layer "time2_vec_4" (type Time2Vec).

in user code:

    File "/tmp/ipykernel_32545/2535887131.py", line 17, in call  *
        sin_trans = tf.math.sin(tf.matmul(tf.ones_like(time_values), self.w) + self.b)  # Modified tf.matmul input

    ValueError: Shape must be rank 2 but is rank 1 for '{{node time2_vec_4/MatMul}} = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false](time2_vec_4/ones_like, time2_vec_4/MatMul/ReadVariableOp)' with input shapes: [?], [2,32].


Call arguments received by layer "time2_vec_4" (type Time2Vec):
  • x=tf.Tensor(shape=(None, 1), dtype=float32)

In [1]:
dense = Dense(64, activation='relu')(time2vec)
outputs = Dense(1, activation='linear')(dense)

model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=Adam(lr=0.001), loss='mse')

# Generate sample data
num_samples = 1000
time_values = tf.random.uniform((num_samples, 1), minval=0, maxval=24)
target_values = 2 * time_values + 5  # Linear relationship for demonstration

# Train the model
model.fit(time_values, target_values, epochs=10, batch_size=32)

# Test the model
test_time_values = tf.random.uniform((10, 1), minval=0, maxval=24)
predictions = model.predict(test_time_values)

print("Predictions:", predictions)


2023-06-12 08:49:37.791373: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


ValueError: Exception encountered when calling layer "time2_vec" (type Time2Vec).

in user code:

    File "/tmp/ipykernel_32545/4024073291.py", line 22, in call  *
        sin_trans = tf.math.sin(tf.matmul(time_values, self.w) + self.b)

    ValueError: Shape must be rank 2 but is rank 1 for '{{node time2_vec/MatMul}} = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false](time2_vec/FloorMod, time2_vec/MatMul/ReadVariableOp)' with input shapes: [?], [1,32].


Call arguments received by layer "time2_vec" (type Time2Vec):
  • x=tf.Tensor(shape=(None, 1), dtype=float32)